In [160]:
import pyodbc
import json
import requests

In [262]:
# pyodbc.dataSources()

In [192]:
driver =  '{Microsoft Access Driver (*.mdb, *.accdb)}'
filePath = r'C:\Users\Danyal\Desktop\New folder\HanvonFaceIDManager.mdb'

In [193]:
myDataSources = pyodbc.dataSources()
access_driver = myDataSources[ 'MS Access Database']

In [194]:
connection = pyodbc.connect(driver = access_driver,dbq = filePath, autocommit=True)

In [195]:
cursor= connection.cursor()

In [196]:
#Reading all the tables from the file
details = []
tables = list(cursor.tables())
for table in tables:
    details.append(table[2]) 

In [210]:
# defining query
query = "SELECT * FROM Attendance WHERE AttendanceID <= 500"
cursor.execute(query)

In [232]:
query2 =  "SELECT * FROM Attendance WHERE AttendanceID <= 1000"
cursor.execute(query2)
query2= cursor.fetchall()

In [211]:
#fetching all rows
rows = cursor.fetchall()

In [217]:
len(rows) #Fetched first 500 rows because there are 15000 rows in one month

500

In [213]:
data = {'items':[]} #Converting data into Json
with open('new.json', 'w') as f:
    for row in rows:
        data['items'].append({'mchdb_pk': row[0],'sensorid':row[1],'userid': row[2],'checktime': row[3],'verifycode':row[4],'checktype':row[5]})

In [216]:
i = 0
for x in data['items']:
    if x['verifycode'] == '01':
        print(x)
        i += 1
        if i % 10 == 0:
            break

{'mchdb_pk': 1, 'sensorid': 1, 'userid': '289', 'checktime': '09-Dec-2017 08:24:53 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 2, 'sensorid': 1, 'userid': '158', 'checktime': '09-Dec-2017 07:46:33 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 5, 'sensorid': 1, 'userid': '2614', 'checktime': '08-Dec-2017 07:33:19 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 7, 'sensorid': 1, 'userid': '625', 'checktime': '09-Dec-2017 08:09:10 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 9, 'sensorid': 1, 'userid': '228', 'checktime': '08-Dec-2017 08:06:21 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 10, 'sensorid': 1, 'userid': '255', 'checktime': '10-Dec-2017 07:48:50 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 12, 'sensorid': 1, 'userid': '3207', 'checktime': '07-Dec-2017 08:04:34 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 13, 'sensorid': 1, 'userid': '3071', 'checktime': '08-Dec-2017 08:03:36 AM', 've

In [219]:
for index, value in enumerate(data['items']):
    if value['verifycode'] == '01':
        r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=1', json = value)
    elif value['verifycode'] == '02':
        r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=2',json = value)
    elif value['verifycode'] == '03':
        r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=3',json = value)

In [226]:
getMaxNum1 = requests.get('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=1') #getting max number from machine 1
getMaxNum2 = requests.get('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=2')#getting max number from machine 2
getMaxNum3 = requests.get('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=3')#getting max number from machine 3

dataJson1 = getMaxNum1.json()
dataJson2= getMaxNum2.json()
dataJson3 = getMaxNum3.json()

#Storing the data  into variable to upload after the last record
for val1 in dataJson1['items']:
    max_value1 = val1['max(mchdb_pk)']

for val2 in dataJson2['items']:
    max_value2 = val2['max(mchdb_pk)'] 

for val3 in dataJson3['items']:
    max_value3 = val3['max(mchdb_pk)'] 

In [269]:
print('Max Value of each sensor id')
print('*' * 10)
print('sensorid "1" :' , max_value1)
print('sensorid "2" :' , max_value2)
print('sensorid "3" :' , max_value3)

Max Value of each sensor id
**********
sensorid "1" : 499
sensorid "2" : 500
sensorid "3" : None


In [234]:
#I was taken the records from 1 - 1000 in query 2 even the last query data included the records of query 1, this will update the record in JSON
data = {'items':[]} #Converting data into Json
with open('new.json', 'w') as f:
    for row in query2:
        data['items'].append({'mchdb_pk': row[0],'sensorid':row[1],'userid': row[2],'checktime': row[3],'verifycode':row[4],'checktype':row[5]})

In [246]:
for val in data['items']: #updating the data after the last record to their locations
    if val['mchdb_pk'] >= max_value1 + 1:
        if val['verifycode'] == '01':
            try:
                r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=1', json = val)
            except Exception as e:
                e
                
for val in data['items']:
    if val['mchdb_pk'] >= max_value2 + 1:  #updating the data after the last record to their locations
        if val['verifycode'] == '02':
            try:
                r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=2', json = val)
            except Exception as e:
                e
            
for val in data['items']:
    if val['mchdb_pk'] >= max_value3 +1 :  #updating the data after the last record to their locations
        if val['verifycode'] == '03':
            try:
                r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=3', json = val)
            except Exception as e:
                e

In [250]:
for val in data['items']: #updating the data after the last record to their locations
    if val['mchdb_pk'] >= max_value2 + 1:
        if val['verifycode'] == '02':
            print(val)
            break
#data was updating after 500

{'mchdb_pk': 501, 'sensorid': 1, 'userid': '2256', 'checktime': '08-Dec-2017 08:12:59 PM', 'verifycode': '02', 'checktype': 'Test'}
